## Overview

This notebook creates database tables in Cassandra, using event data from a music app. The tables are designed according to business questions the product owner may have. We will create the data tables with these queries in mind.

# Part I. ETL Pipeline for Pre-Processing the Files

## The following code preprocesses the files necessary for ingestion into Cassandra:

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Cassandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Loading into Cassandra data tables

## We will work with the CSV file <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS music
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    ''')
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
session.set_keyspace('music')

### With Apache Cassandra database tables are modelled on the queries we want to run.

In this notebook, we look at 3 hypothetical scenarios. For each scenario, we model a data table after the query we plan to execute

## Query 1:

Imagine that the product owner wants to know the artist, song title and length of a song for a given session and the item in session.

Specifically: 
### 1. The artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

To query efficiently, we need to designate session_id and item_in_session as the primary key. This will lead to data tables being partitioned under session_id and item_in_session numbers

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_item"
query = query + "(sessionId bigint, itemInSession bigint, artist text, song text, length float, PRIMARY KEY ((sessionId, itemInSession), artist, song, length))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = '''SELECT artist, song, length FROM session_item 
           WHERE sessionId=338 AND itemInSession=4
        '''
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2:

Next, the product owner wants to know the artist, song title and user's name for a given user and session, with the song title sorted by item_in_session.

Specifically: 
### 2. Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We designate user_id and session_id as the primary key. This will lead to data tables being partitioned under session_id and item_in_session numbers. We use item_in_session and song_title as clustering columns. This will sort the songs by item_in_session first, followed by the alphabetical order of the song title and finally artist name.

In [12]:
query = "CREATE TABLE IF NOT EXISTS user_session"
query = query + "(userId bigint, sessionId bigint, itemInSession bigint, artist text, song text, user text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, user)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

In [14]:
query = "SELECT artist, song, user FROM user_session WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3:

Finally, the product owner wants to know the user who listened to a particular song.

Specifically: 
### 3. User's name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

We designate song_title as the primary key , to partition data by songs, allowing efficient querying by WHERE clause based on song titles. The user column is used as a clustering column, to sort users who listened to a particular song in alphabetical order.

In [15]:
query = "CREATE TABLE IF NOT EXISTS user_song"
query = query + "(song text, user text, PRIMARY KEY (song))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, user)"
        query = query + " VALUES (%s, %s)"
        session.execute(query, (line[9], line[1] + ' ' + line[4]))

In [17]:
query = "SELECT user FROM user_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.user)

Sara Johnson


## With that, we are done. To free up storage, drop the tables as we no longer have any need for them. Then, close the connection to the session we have.

In [18]:
drop_query_0 = "drop table session_item"
drop_query_1 = "drop table user_session"
drop_query_2 = "drop table user_song"
try:
    session.execute(drop_query_0)
    session.execute(drop_query_1)
    session.execute(drop_query_2)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()